## Краулер для Politru_lectures

Импортируем все необходимые методы для последующей работы

In [30]:
import re
import urllib.request
import bs4 as bs
import pandas as pd
import os

Создаем пустой датафрейм. Он пригодится нам в дальнейшем для сортировки данных

In [79]:
ready_df = pd.DataFrame()

## Скачиваем страницы
Функция для скачивания контента по ссылке в виде файла. Аргумент функции - ссылка. Результат работы кода функции - файл в директории "Pages". Уникальное имя файла генерируется за счет части url (на основе анализа ссылок на статьи в Indicator.ru) за счет замены / на _ и отсекания общей части для всех url.

In [3]:
def download(url):
    source = urllib.request.urlopen(url).read().decode('utf-8')
    file_name = url.split('http://polit.ru/article/')[1].replace('/','_')
    fl = open('pages_politru_lectures/' + file_name, 'w')
    fl.write(str(source))
    fl.close()
    

Функция находит в словаре ключ по значению

In [80]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if [value] in v), None)

Функция принимает на вход путь к файлу (из ранее сгенерированной директории "Pages"). При помощи библиотеки Beautiful Soup на основании предварительного анализа структуры статей сайта находим элементы статьи: 
- путь до текстового файла на Яндекс-диске
- источник
- url
- дату (дату берем из ссылки на статью при помощи регулярных выражений)
- заголовок
- подзаголовок
- автор 
- рубрика
- тэги
- жанр

Полученные данные добавляем в заранее заготовленный пустой датафрейм; каждая колонка соответсвует вышеперечисленным элементам

In [81]:
def clear(file_path, dictionary):
    
    global ready_df
    
    with open(file_path) as fl:
        source = fl.read()
    
    soup_article = bs.BeautifulSoup(source, 'lxml')
    
    clear_p = ''
    
    for paragraph in soup_article.find_all('p'):
        clear_p = clear_p + paragraph.text
    
    head_title = soup_article.find('h1', class_='title').text

    rubrics = ''
    _rubrics = soup_article.find_all('div', class_= 'tags-subject')
    for rub_href in _rubrics:
        rub_tag = rub_href.select('a[href]')
        for tag in rub_tag:
            rubrics = rubrics + '_' + tag.get_text().lower()     
    

    raw_date = soup_article.find('div', class_='date')
    pre_date = re.search(r'.*([0-9]{2}\s[а-я]+\s[0-9]{4}).*', str(raw_date))
    date = pre_date.group(1)
    
    source = 'polit.ru/lectures'
    
    f_name = file_path.replace('pages_politru_lectures/','')
    name = f_name.replace('./', '')
    url = 'https://polit.ru/article/' + name.replace('_','/')
    path = 'politru_lectures/politru_lectures_clear_text/' + name + '.txt'
    genre = 'Лекции'
    
    value = '/article/' + name.replace('_', '/')
    author = find_key(dictionary, value)
    
    subtitle = ''
    big_rubrics = ''

    df = pd.DataFrame([[path, source, url, date, head_title, subtitle, author, big_rubrics, rubrics, genre]],
                      columns=['path', 'source', 'url', 'date','title','subtitle','author','rubrics','tags', 'genre'])
    
    save_clear_text(clear_p, file_path.replace('pages_politru_lectures/',''))
    
    ready_df = ready_df.append(df)

## Записываем тексты статей в отдельные файлы
Функция принимает в качестве аргумента текст и имя файла для записи (папка clear_text создана заранее). Данная функция понадобится нам в следующей функции.

In [4]:
def save_clear_text(text, file_name):
    f = open('politru_lectures_clear_text/'+file_name + '.txt', 'w')
    f.write(text)
    f.close()

## Код
К адресу сайта При помощи Beuatiful Soup выделяем адреса ссылок по лекторам. Циклом при помощи написанной выше функции скачивания проходим по всем ссылкам лекций и скачиваем их в папку "Pages_politru_lectures".

In [5]:
lectures_by_authors = urllib.request.urlopen('http://polit.ru/lector/').read().decode('utf-8')
soup_links = bs.BeautifulSoup(lectures_by_authors, 'lxml') 

links = []

for author_link in soup_links.find_all('h3', class_ = 'title'):
    get_href = str(author_link.select('a[href^="/lector/"]')[0])
     
    if re.search(r'/lector/\w+/', get_href) == None:
        continue
    else:
        clear_href = re.findall(r'/lector/\w+/', get_href)[0]
        links.append('http://polit.ru' + clear_href)    

Создаем словарь, в котором ключ - имя автора(в формате строки), значения - адреса на лекции этого автора(список строк)

In [44]:
article_links = []
authors = {}

for author_dir in links:
    author_page = urllib.request.urlopen(author_dir).read().decode('utf-8')
    soup_author_page = bs.BeautifulSoup(author_page, 'lxml')
    
    author = soup_author_page.find('h1', class_='title').text
    
    
    for article_field in soup_author_page.find_all('div', class_ = 'article-tag-result stop'):
        clear_href = article_field.select('a[href^="/article/"]')
        
        for raw_link in clear_href:
            article_links.append(re.findall(r'/article/[0-9]{4}/[0-9]{2}/[0-9]{2}/\w+/', 
                                            str(raw_link)))
            authors.setdefault(author, []).append(re.findall(r'/article/[0-9]{4}/[0-9]{2}/[0-9]{2}/\w+/', 
                                            str(raw_link)))
            
            

Скачиваем страницы написанной ранее функцией downlaod

In [107]:
for art_url in article_links:
    download('http://polit.ru' + art_url[0])

Сохраняем очищенные тексты при помощи написанной выше функции clear

In [82]:
start_path = './pages_politru_lectures/'

for path, dirs, files in os.walk(start_path):

    for fname in files:
        if not fname.startswith('.'):
            clear(start_path+fname, authors)


Сохраняем метаданные в формате csv

In [86]:
ready_df.to_csv('politru_lectures.сsv', sep=',')